In [4]:
import sys
import math
import functools
import datetime

In [24]:
def flatten_boxes(boxes):
    exclusion = ['A2 B', 'B1 L', 'B2 L', 'B2 B']
    return list(filter(None, functools.reduce(
        lambda m, box: m + list(map(lambda side: f'{box[0]} {side}' if f'{box[0]} {side}' not in exclusion else None, list(box[1]))), boxes, [])))
boxes = ['A1 LTRB', 'A2 LTRB', 'B1 LTRB', 'B2 LTRB']
print(flatten_boxes(boxes))

In [25]:
def remove_move(boxes, move):
    move_box, move_side = move.split(' ')
    clone = boxes[:]

    opposites = {
        'T': 'B',
        'B': 'T',
        'L': 'R',
        'R': 'L'
    }
    
    for i, box in enumerate(clone):
        a = move_side == 'R' and move_box[1] == box[0][1] and ord(move_box[0])+1 == ord(box[0][0])
        b = move_side == 'T' and move_box[0] == box[0][0] and ord(move_box[1])+1 == ord(box[0][1])
        if box[0] == move_box:
            new_sides = box[1].replace(move_side, '')
            clone[i] = (box[0], new_sides) if new_sides != '' else None
        elif a or b:
            new_sides = box[1].replace(opposites[move_side],"")
            clone[i] = (box[0], new_sides) if new_sides != '' else None
            
    nonone = list(filter(lambda b: not (b is None), clone))

    return nonone

In [26]:
def is_candidate_closed_by(box, move):
    move_box, move_side = move.split(' ')
    a = move_side == 'R' and move_box[1] == box[0][1] and ord(move_box[0])+1 == ord(box[0][0])
    b = move_side == 'T' and move_box[0] == box[0][0] and ord(move_box[1])+1 == ord(box[0][1])
    opposites = {
        'T': 'B',
        'B': 'T',
        'L': 'R',
        'R': 'L'
    }

    if box[0] == move_box and box[1] == move_side: # closes box
        return True
    if a: # closes same cloumn
        return True
    elif b: # closes same row
        return True

    return False

In [27]:
def closed_boxes(boxes, move):
    candidates = list(filter(lambda b: len(b[1]) == 1, boxes))
    closed = list(filter(lambda b: is_candidate_closed_by(b, move), candidates))

    return len(closed)

In [28]:
# boxes [('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')] - primera optimizacion, eliminar jugadas repetidas
# Hecho, lmao
def alfa_beta(boxes, max_turn, max_score, min_score, depth, alpha, beta):
    if depth == 0 or len(boxes) == 0: # segunda optimizacion, cortar en profundidad y evaluar
        return max_score - min_score
        
    if max_turn:
        max_eval = -10000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            closedBoxes = closed_boxes(boxes, move)
            print(closedBoxes)# tercera optimizacion, usar numero de cajas cerradas para ordenar movimientos
            new_max_score = max_score + closedBoxes 
            max_eval = max(max_eval, alfa_beta(new_moves, not max_turn, new_max_score, min_score, depth - 1, alpha, beta))
            alpha = max(alpha, max_eval) # cuarta optimizacion, usar poda alfa-beta.Hecho, lmao
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = 10000
        children =  flatten_boxes(boxes)
        for move in children:
            new_moves = remove_move(boxes, move)
            new_min_score = min_score + closed_boxes(boxes, move)
            min_eval = min(min_eval, alfa_beta(new_moves, not max_turn, max_score, new_min_score, depth - 1, alpha, beta))
            beta = min(beta, min_eval)
            if beta <= alpha:
                break
        return min_eval
alfa_beta([('A1', 'LTRB'), ('A2', 'LTRB'), ('B1', 'LTRB'), ('B2', 'LTRB')], True, 0, 0, 3, -10000, 10000)

0

In [ ]:
def move(boxes, player_score, opponent_score):
  max_eval = -1000
  children =  flatten_boxes(boxes)
  best_move = None
  for move in children:
      new_moves = remove_move(boxes, move)
      new_max_score = player_score + closed_boxes(boxes, move)
      rating = alfa_beta(new_moves, True, new_max_score, opponent_score, 3, -1000, 1000)
      if rating > max_eval:
        max_eval = rating
        best_move = move

  return best_move

In [ ]:
#game loop
while True:
    player_score, opponent_score = [int(i) for i in input().split()]

    boxes = []
    for i in range(int(input())):
        boxes.append(input())

    print(f'{move(boxes, player_score, opponent_score)} MSG {boxes}')